<a href="https://colab.research.google.com/github/BurgundyIsAPublicEnemy/ORLIAn-source/blob/main/ORLIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
IMAGEFILES = '/content/drive/MyDrive/ORLIAn /image_data'
TABULARTRAINING = '/content/drive/MyDrive/ORLIAn /KORL_avatar_train.csv'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

count = 0
import os
for dirname, _, filenames in os.walk(IMAGEFILES):
    for filename in filenames:
        count = count + 1
print(count , " files detected")

1421  files detected


In [ ]:
train_df = pd.read_csv(TABULARTRAINING)

NameError: ignored

In [ ]:
train_df.head()

,Patient_ID,OMS,Sexe,Age,Date_biopsie,Age_diag,Date_derniere_nouvelles,Deces,Recidive,localisation,RNA_scope,T,N,M,Tabac,Alcool,OS
0,824635,0,1,1965-05-11,2014-03-28,48,2019-11-06,0,0,Oropharynx,2,1,2a,0,3,1,61
1,829316,1,0,1942-12-06,2014-01-01,71,2018-01-09,1,0,Oropharynx,2,3,2b,0,0,1,56
2,82dfbb,1,1,1963-02-05,2012-03-28,48,2020-08-27,1,0,Oropharynx,1,4,2,0,0,1,101
3,832c5f,1,1,1957-11-27,2013-10-25,55,2014-10-25,1,1,Oropharynx,2,2,2b,0,2,1,12
4,837956,0,1,1959-03-15,2012-06-09,53,2021-04-06,0,0,Oropharynx,1,4,1,0,3,1,103


# Convert the times in to datetimes

In [ ]:
train_df.describe()

,OMS,Sexe,Age_diag,Deces,Recidive,RNA_scope,T,M,Tabac,Alcool,OS
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.00000,47.000000,47.000000,47.000000,47.00000
mean,0.893617,0.829787,61.212766,0.297872,0.212766,1.531915,3.06383,0.063830,1.638298,0.978723,47.12766
std,0.840136,0.379883,10.127312,0.462267,0.413688,0.584252,1.00875,0.247092,1.358156,0.145865,30.09739
min,0.000000,0.000000,47.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,1.00000
25%,0.000000,1.000000,53.500000,0.000000,0.000000,1.000000,2.00000,0.000000,0.000000,1.000000,19.00000
50%,1.000000,1.000000,58.000000,0.000000,0.000000,2.000000,3.00000,0.000000,2.000000,1.000000,56.00000
75%,1.000000,1.000000,69.000000,1.000000,0.000000,2.000000,4.00000,0.000000,3.000000,1.000000,72.00000
max,3.000000,1.000000,83.000000,1.000000,1.000000,2.000000,4.00000,1.000000,3.000000,1.000000,103.00000


In [ ]:
pd.to_datetime(train_df['Date_derniere_nouvelles'])
pd.to_datetime(train_df['Age'])

0    1965-05-11
1    1942-12-06
2    1963-02-05
3    1957-11-27
4    1959-03-15
5    1956-07-31
6    1939-10-12
7    1946-01-27
8    1966-02-02
9    1954-06-22
10   1959-04-18
11   1954-03-28
12   1934-12-27
13   1964-01-03
14   1947-07-15
15   1946-07-15
16   1933-05-12
17   1958-04-23
18   1948-02-27
19   1928-04-07
20   1957-01-15
21   1938-12-07
22   1938-01-12
23   1944-06-05
24   1965-05-28
25   1946-03-20
26   1957-01-12
27   1940-09-04
28   1955-08-29
29   1941-07-22
30   1953-10-19
31   1963-10-31
32   1963-11-14
33   1958-05-12
34   1959-10-16
35   1958-10-15
36   1946-03-21
37   1945-10-10
38   1940-10-04
39   1951-06-07
40   1955-02-15
41   1954-11-28
42   1960-06-15
43   1958-07-21
44   1948-02-06
45   1931-11-26
46   1961-02-13
Name: Age, dtype: datetime64[ns]

In [ ]:
import datetime
cur = np.datetime64('today')

agedDF = pd.to_datetime(train_df['Age']).apply(lambda x : (cur - x)  )
print(agedDF)

0    20627 days
1    28819 days
2    21453 days
3    23349 days
4    22876 days
5    23833 days
6    29970 days
7    27671 days
8    20360 days
9    24603 days
10   22842 days
11   24689 days
12   31720 days
13   21121 days
14   27137 days
15   27502 days
16   32314 days
17   23202 days
18   26910 days
19   34175 days
20   23665 days
21   30279 days
22   30608 days
23   28272 days
24   20610 days
25   27619 days
26   23668 days
27   29642 days
28   24170 days
29   29321 days
30   24849 days
31   21185 days
32   21171 days
33   23183 days
34   22661 days
35   23027 days
36   27618 days
37   27780 days
38   29612 days
39   25714 days
40   24365 days
41   24444 days
42   22418 days
43   23113 days
44   26931 days
45   32847 days
46   22175 days
Name: Age, dtype: timedelta64[ns]


In [ ]:
train_df.head()

,Patient_ID,OMS,Sexe,Age,Date_biopsie,Age_diag,Date_derniere_nouvelles,Deces,Recidive,localisation,RNA_scope,T,N,M,Tabac,Alcool,OS
0,824635,0,1,1965-05-11,2014-03-28,48,2019-11-06,0,0,Oropharynx,2,1,2a,0,3,1,61
1,829316,1,0,1942-12-06,2014-01-01,71,2018-01-09,1,0,Oropharynx,2,3,2b,0,0,1,56
2,82dfbb,1,1,1963-02-05,2012-03-28,48,2020-08-27,1,0,Oropharynx,1,4,2,0,0,1,101
3,832c5f,1,1,1957-11-27,2013-10-25,55,2014-10-25,1,1,Oropharynx,2,2,2b,0,2,1,12
4,837956,0,1,1959-03-15,2012-06-09,53,2021-04-06,0,0,Oropharynx,1,4,1,0,3,1,103


In [ ]:
train_df.isnull( ).sum( )

Patient_ID                 0
OMS                        0
Sexe                       0
Age                        0
Date_biopsie               0
Age_diag                   0
Date_derniere_nouvelles    0
Deces                      0
Recidive                   0
localisation               0
RNA_scope                  0
T                          0
N                          0
M                          0
Tabac                      0
Alcool                     0
OS                         0
dtype: int64